# Keras Implementation - Human Data Analytics Project 

Import of all modules

In [193]:
import numpy as np
import scipy.io

from keras.layers import Conv1D, BatchNormalization, Dropout, LeakyReLU, Flatten
from keras.models import Model, Sequential

from keras.optimizers import Adam

from sklearn.preprocessing import OneHotEncoder

import keras.backend as K
K.set_image_data_format('channels_last')

%matplotlib inline

# Preprocessing Section

In [196]:
x1 = scipy.io.loadmat("data_temp/S1-ADL1", mdict={'filled_features':'features', 'labels':'labels'})
x2 = scipy.io.loadmat("data_temp/S1-ADL2", mdict={'filled_features':'features', 'labels':'labels'})
x3 = scipy.io.loadmat("data_temp/S1-ADL3", mdict={'filled_features':'features', 'labels':'labels'})
x4 = scipy.io.loadmat("data_temp/S1-ADL4", mdict={'filled_features':'features', 'labels':'labels'})
x5 = scipy.io.loadmat("data_temp/S1-ADL5", mdict={'filled_features':'features', 'labels':'labels'})

In [202]:
X_train = np.concatenate((x1['filled_features'],x2['filled_features'],x3['filled_features']),axis=0)
X_test = np.concatenate((x4['filled_features'],x5['filled_features']),axis=0)

Y_train = np.concatenate((x1['labels'][:,0],x2['labels'][:,0],x3['labels'][:,0]),axis=0) # here we take just the first column because we want to fulfill the first task
Y_test = np.concatenate((x4['labels'][:,0],x5['labels'][:,0]),axis=0)

# Initial decision to overcome the problem of entire missing columns
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)


X_train shape:  (104973, 113)
X_test shape:  (57536, 113)


In [206]:
onehot_encoder = OneHotEncoder(sparse=False)
YOH_train = onehot_encoder.fit_transform(Y_train.reshape(-1, 1))
YOH_test = onehot_encoder.fit_transform(Y_test.reshape(-1, 1))

print("YOH_train shape: ", YOH_train.shape)
print("YOH_test shape: ", YOH_test.shape)

YOH_train shape:  (104973, 5)
YOH_test shape:  (57536, 5)


In [209]:
window_size = 30
stride = int(window_size / 2)
num_features = X_train.shape[1]
samples, classes = YOH_train.shape
print("Number of samples: ", samples, "\nNumber of classes: ", classes)

windows = int(samples // stride) - 1
print("Number of windows: ", windows)

Number of samples:  104973 
Number of classes:  5
Number of windows:  6997


In [210]:
cube = np.zeros([windows, window_size, num_features])
cube.shape

(6997, 30, 113)

In [211]:
lab_cum = np.zeros([windows])
lab_cum.shape

(6997,)

In [212]:
for w in range(windows):
    index = int(w * stride)
    cube[w,:,:] = X_train[index:index+window_size, :].reshape((window_size,num_features))
    l = YOH_train[index:index+window_size,:] # shape 50 x 5
    #print(np.sum(l, axis=0),np.argmax(np.sum(l, axis=0)))
    lab_cum[w] = np.argmax(np.sum(l, axis=0))

In [213]:
#print('cumulative labels: ', lab_cum.shape, type(lab_cum), "\n", lab_cum)
YOH_train_final = onehot_encoder.fit_transform(lab_cum.reshape(-1, 1))
print(YOH_train_final)

[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [222]:
def TestModel(input_shape):
    """ 
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns: 
    model -- a Model() instance in Keras
    """
    
    model = Sequential()
    model.add(Conv1D(filters = 16,
                    kernel_size=3,
                    strides=1,
                    padding='same',
                    input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters = 32,
                    kernel_size=5,
                    strides=1,
                    padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    
    model.add(Flatten())

    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    model.summary()
    ### END CODE HERE ###
    
    return model

In [223]:
model_test = TestModel((30,113))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_23 (Conv1D)           (None, 30, 16)            5440      
_________________________________________________________________
batch_normalization_32 (Batc (None, 30, 16)            64        
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 30, 16)            0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 30, 16)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 30, 32)            2592      
_________________________________________________________________
batch_normalization_33 (Batc (None, 30, 32)            128       
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 30, 32)            0         
__________

In [227]:
opt = Adam(lr=0.0001)
model_test.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model_test.fit(x = cube, y = YOH_train_final, epochs = 50, batch_size = 40)

Epoch 1/50
6997/6997 [==============================] - 4s 556us/step - loss: 1.3494 - acc: 0.4419
Epoch 2/50
6997/6997 [==============================] - 2s 355us/step - loss: 1.3436 - acc: 0.4463
Epoch 3/50
6997/6997 [==============================] - 2s 346us/step - loss: 1.3464 - acc: 0.4450
Epoch 4/50
6997/6997 [==============================] - 2s 353us/step - loss: 1.3499 - acc: 0.4442
Epoch 5/50
6997/6997 [==============================] - 2s 347us/step - loss: 1.3443 - acc: 0.4448
Epoch 6/50
 720/6997 [==>...........................] - ETA: 2s - loss: 1.3545 - acc: 0.4431

In [126]:
lab_cum

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])